In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'outside_diameter_metric'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [9]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['OutsideDiameter']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

4467


In [26]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, OutsideDiameter, name, matches, mm]
Index: []

In [11]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [12]:
# df['MaxPressure'].explode().value_counts()

In [37]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+.?\d*\/?\d*mm)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
# df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+.?\d*\/?\d*mm)|()'
df['matches']=df['OutsideDiameter'].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(mm)|()'
df['mm']=df['OutsideDiameter'].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x)))                                          
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
# custom[curation_col].explode().value_counts()
# # custom[custom[curation_col].astype(str)==r'300 150   ']
custom[0:500]

179


buckets external_id  \
3625  Compressors Accessories & Replacement Parts       51583   
3764  Compressors Accessories & Replacement Parts       51367   
3776  Compressors Accessories & Replacement Parts       51371   
3788  Compressors Accessories & Replacement Parts       51478   
3800  Compressors Accessories & Replacement Parts       51477   
3867  Compressors Accessories & Replacement Parts       51369   
3901  Compressors Accessories & Replacement Parts       51475   
3956  Compressors Accessories & Replacement Parts       51547   
3967  Compressors Accessories & Replacement Parts       51521   
3978  Compressors Accessories & Replacement Parts       51549   
3989  Compressors Accessories & Replacement Parts       51519   
4000  Compressors Accessories & Replacement Parts       51541   
4011  Compressors Accessories & Replacement Parts       51573   
4022  Compressors Accessories & Replacement Parts       51518   
4033  Compressors Accessories & Replacement Parts       51540   
4044  Compressors Accessories & Replacement Parts       51522   
4055  Compressors Accessories & Replacement Parts       51542   
4064  Compressors Accessories & Replacement Parts       51546   
4065  Compressors Accessories & Replacement Parts       51544   
4066  Compressors Accessories & Replacement Parts       51427   
4067  Compressors Accessories & Replacement Parts       51525   
4068  Compressors Accessories & Replacement Parts       51517   
4069  Compressors Accessories & Replacement Parts       51570   
4070  Compressors Accessories & Replacement Parts       51520   
4071  Compressors Accessories & Replacement Parts       51523   
4072  Compressors Accessories & Replacement Parts       51581   
4073  Compressors Accessories & Replacement Parts       51571   
4074  Compressors Accessories & Replacement Parts       51578   
4075  Compressors Accessories & Replacement Parts       51516   
4076  Compressors Accessories & Replacement Parts       51579   
4077  Compressors Accessories & Replacement Parts       51576   
4078  Compressors Accessories & Replacement Parts       51584   
4079  Compressors Accessories & Replacement Parts       51596   
4080  Compressors Accessories & Replacement Parts       51433   
4081  Compressors Accessories & Replacement Parts       51426   
4082  Compressors Accessories & Replacement Parts       51575   
4083  Compressors Accessories & Replacement Parts       51422   
4084  Compressors Accessories & Replacement Parts       51424   
4085  Compressors Accessories & Replacement Parts       51515   
4086  Compressors Accessories & Replacement Parts       51413   
4087  Compressors Accessories & Replacement Parts       51420   
4088  Compressors Accessories & Replacement Parts       51434   
4089  Compressors Accessories & Replacement Parts       51586   
4090  Compressors Accessories & Replacement Parts       51425   
4091  Compressors Accessories & Replacement Parts       51514   
4092  Compressors Accessories & Replacement Parts       51593   
4093  Compressors Accessories & Replacement Parts       51421   
4094  Compressors Accessories & Replacement Parts       51589   
4095  Compressors Accessories & Replacement Parts       51550   
4096  Compressors Accessories & Replacement Parts       51412   
4097  Compressors Accessories & Replacement Parts       51594   
4098  Compressors Accessories & Replacement Parts       51443   
4099  Compressors Accessories & Replacement Parts       51432   
4100  Compressors Accessories & Replacement Parts       51590   
4101  Compressors Accessories & Replacement Parts       51471   
4102  Compressors Accessories & Replacement Parts       51447   
4103  Compressors Accessories & Replacement Parts       51454   
4104  Compressors Accessories & Replacement Parts       51406   
4105  Compressors Accessories & Replacement Parts       51423   
4106  Compressors Accessories & Replacement Parts       51439   
4107  Compressors Accessories & Replacement Parts       51437   
4108  Compre

In [31]:
print(len(df[df['name'].astype(str)!='[]']))
print(len(df[df['name'].astype(str)=='[]']))
df[df['name'].astype(str)=='[]'][0:500]

601
3866


buckets external_id  \
0    Compressors Accessories & Replacement Parts       40511   
1    Compressors Accessories & Replacement Parts       40490   
2    Compressors Accessories & Replacement Parts       40492   
3    Compressors Accessories & Replacement Parts       40491   
4    Compressors Accessories & Replacement Parts       40493   
5    Compressors Accessories & Replacement Parts       40494   
6    Compressors Accessories & Replacement Parts       40495   
7    Compressors Accessories & Replacement Parts       40504   
8    Compressors Accessories & Replacement Parts      404951   
9    Compressors Accessories & Replacement Parts       40497   
10   Compressors Accessories & Replacement Parts      404952   
11   Compressors Accessories & Replacement Parts      404971   
12   Compressors Accessories & Replacement Parts     40497EZ   
13   Compressors Accessories & Replacement Parts       40498   
14   Compressors Accessories & Replacement Parts       40500   
15   Compressors Accessories & Replacement Parts     40500EZ   
16   Compressors Accessories & Replacement Parts       40502   
17   Compressors Accessories & Replacement Parts      405061   
18   Compressors Accessories & Replacement Parts      405142   
19   Compressors Accessories & Replacement Parts      405051   
20   Compressors Accessories & Replacement Parts       40503   
21   Compressors Accessories & Replacement Parts       40508   
22   Compressors Accessories & Replacement Parts       40509   
23   Compressors Accessories & Replacement Parts       40510   
24   Compressors Accessories & Replacement Parts       40514   
25   Compressors Accessories & Replacement Parts      405143   
26   Compressors Accessories & Replacement Parts       40501   
27   Compressors Accessories & Replacement Parts       40515   
28   Compressors Accessories & Replacement Parts      405151   
29   Compressors Accessories & Replacement Parts      405154   
30   Compressors Accessories & Replacement Parts      405153   
31   Compressors Accessories & Replacement Parts      405152   
32   Compressors Accessories & Replacement Parts      405155   
33   Compressors Accessories & Replacement Parts       40512   
34   Compressors Accessories & Replacement Parts       40513   
35   Compressors Accessories & Replacement Parts       40505   
36   Compressors Accessories & Replacement Parts       40545   
37   Compressors Accessories & Replacement Parts      405621   
38   Compressors Accessories & Replacement Parts      405622   
39   Compressors Accessories & Replacement Parts       40567   
40   Compressors Accessories & Replacement Parts      405642   
41   Compressors Accessories & Replacement Parts       40564   
42   Compressors Accessories & Replacement Parts       40568   
43   Compressors Accessories & Replacement Parts      405761   
44   Compressors Accessories & Replacement Parts      405771   
45   Compressors Accessories & Replacement Parts      405772   
46   Compressors Accessories & Replacement Parts      405741   
47   Compressors Accessories & Replacement Parts       40578   
48   Compressors Accessories & Replacement Parts      405752   
49   Compressors Accessories & Replacement Parts       40575   
50   Compressors Accessories & Replacement Parts      405781   
51   Compressors Accessories & Replacement Parts      405783   
52   Compressors Accessories & Replacement Parts      405784   
53   Compressors Accessories & Replacement Parts      405785   
54   Compressors Accessories & Replacement Parts      405786   
55   Compressors Accessories & Replacement Parts      405787   
56   Compressors Accessories & Replacement Parts      405782   
57   Compressors Accessories & Replacement Parts       40579   
58   Compressors Accessories & Replacement Parts       40577   
59   Compressors Accessories & Replacement Parts      405759   
60   Compressors Accessories & Replacement Parts       40576   
61   Compressors Accessories & Replacement Parts      405911   
62   Comp

In [38]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [38]:
two=df[(df['matches'].astype(str)=='[]')]
print(len(two[two['name'].astype(str)!='[]']))
two=two[two['name'].astype(str)!='[]']
two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
two[curation_col].explode().value_counts()
matchtwo=two[['external_id',curation_col]]

1034


In [24]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [33]:
na=df[(df['name'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

3866


In [34]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 